# Report

In [1]:
%cd ..

/Users/andersvandvik/Repositories/project-thesis


In [2]:
import os
import sys
import argparse
import data
import folium
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pprint import pprint
from IPython.display import display, HTML
import arc_flow.preprocessing.helpers as hlp
%matplotlib inline

In [3]:
CONFIG_FILE = '.config_ipynb'
path = f'{data.PROJECT_DIR_PATH}/report'

with open(f'{path}/{CONFIG_FILE}') as f:
    sys.argv = f.read().split()

if sys.argv:
    parser = argparse.ArgumentParser()
    parser.add_argument('file_path', type=str, default=2)
    args = parser.parse_args()
    file_path = args.file_path
else:
    file_path = '/Users/andersvandvik/Repositories/project-thesis/output/local/results/O4-I3-C1-V1-WS0.json'

In [4]:
inst_to_coord = {
    0: [60.87, 4.71],
    1: [60.64, 3.72],
    2: [60.77, 3.50],
    3: [60.88, 3.60],
    4: [60.74, 3.61],
    5: [60.95, 3.58],
    6: [60.85, 3.62],
    7: [60.89, 3.67],
    8: [60.48, 2.82],
    9: [60.48, 2.82],
    10: [60.60, 2.77],
    11: [60.70, 2.93],
    12: [60.70, 2.93],
    13: [60.38, 2.79],
    14: [60.08, 2.63],
    15: [61.07, 2.50],
    16: [61.04, 2.34],
    17: [61.04, 2.34],
    18: [60.78, 2.89],
    19: [60.85, 2.65],
    20: [60.85, 2.65],
    21: [61.25, 1.85],
    22: [61.20, 1.82],
    23: [61.29, 1.90],
    24: [61.17, 2.18],
    25: [61.20, 2.20],
    26: [61.20, 2.27],
    27: [60.90, 3.81]
}

vessel_to_color = {
    0: 'blue',
    1: 'green',
    2: 'yellow',
    3: 'violet',
    4: 'orange',
    5: 'red'
}

In [5]:
def summarize_voyage_orders(voyages):
    column_names = ['Vessel', 'Start node', 'End node', 
                    'Start time', 'Arrival time', 'End time', 'Speed',
                    'Delivery load', 'Pickup load',
                    'Arc fuel cost', 'Arc charter cost']
    rows = []
    for vessel in voyages.keys():
        start_nodes = [int(key) for key in voyages[vessel].keys()]
        start_nodes.sort()
        start_node, final_start_node = start_nodes[0], start_nodes[-1]
        while start_node <= final_start_node:
            leg_info = voyages[vessel][str(start_node)]
            end_node = leg_info[0]
            start_time, arr_time, end_time = leg_info[1]
            speed = leg_info[2]
            delivery_load, pickup_load = leg_info[3]
            arc_fuel_cost, arc_charter_cost = leg_info[4]
            row = [int(vessel), int(start_node), end_node, 
                   start_time, arr_time, end_time, speed,
                   float(delivery_load), float(pickup_load), 
                   arc_fuel_cost, arc_charter_cost]
            rows.append(row)
            start_node = int(end_node)
    df = pd.DataFrame(rows, columns=column_names)
    return df

def summarize_voyage_insts(voyage_by_orders, order_info, end_depot_node):
    column_names = ['Vessel', 'Start inst', 'End inst', 
                    'Start time', 'Arrival time', 'End time', 'Speed',
                    'Delivery load', 'Pickup load',
                    'Arc fuel cost', 'Arc charter cost']
    vessels = set(voyage_by_orders.loc[:, 'Vessel'])
    rows = []
    for vessel in vessels:
        vessel_rows = []
        for index, row in voyage_by_orders.loc[voyage_by_orders['Vessel'] == vessel].iterrows():
            start_node, end_node = int(row['Start node']), int(row['End node'])
            start_inst = 0 if start_node == 0 else int(order_info.loc[order_info['Node'] 
                                                                      == start_node]['Installation'])
            end_inst = 0 if end_node == end_depot_node else int(order_info.loc[order_info['Node'] 
                                                                      == end_node]['Installation'])
            start_time, arr_time, end_time = row['Start time'], row['Arrival time'], row['End time']
            speed = row['Speed']
            delivery_load, pickup_load = row['Delivery load'], row['Pickup load']
            fuel_cost, charter_cost = row['Arc fuel cost'], row['Arc charter cost']
            row = [vessel, start_inst, end_inst, start_time, arr_time, end_time, speed,
                   delivery_load, pickup_load, fuel_cost, charter_cost]
            vessel_rows.append(row)
        df_vessel = pd.DataFrame(vessel_rows, columns=column_names)
        end_insts = set(df_vessel.loc[:, 'End inst'])
        vessel_rows = []
        for end_inst in end_insts:
            sub_df = df_vessel.loc[df_vessel['End inst'] == end_inst]
            start_inst = sub_df['Start inst'].iloc[0]
            start_time = int(sub_df['Start time'].min())
            arr_time = int(sub_df['Arrival time'].min())
            end_time = int(sub_df['End time'].max())
            speed = sub_df['Speed'].max()
            delivery_load, pickup_load = sub_df['Delivery load'].max(), sub_df['Pickup load'].max()
            fuel_cost, charter_cost = sub_df['Arc fuel cost'].sum(), sub_df['Arc charter cost'].sum()
            row = [vessel, start_inst, end_inst, start_time, arr_time, end_time, speed,
                   delivery_load, pickup_load, fuel_cost, charter_cost]
            vessel_rows.append(row)
            
        # Sort vessel_rows by departure_time column
        rows.extend(vessel_rows)
    df = pd.DataFrame(rows, columns=column_names)
    return df

def organize_solution_attributes(postponed_orders, serviced_orders, fuel_costs, charter_costs, 
                                 penalty_costs, model_runtime, preprocess_runtime):
    index_names = ['Postponed orders', 'Serviced orders', 
                   'Fuel costs', 'Charter costs', 'Penalty costs', 
                   'Model runtime', 'Preprocess runtime']
    column_names = ['Value']
    df = pd.DataFrame([[postponed_orders], [serviced_orders], [fuel_costs], [charter_costs], 
                       [penalty_costs], [model_runtime], [preprocess_runtime]], 
                      columns=column_names, index=index_names)
    return df

def organize_instance_info(fleet_size, inst_ordering, number_of_insts, weather_scenario, number_of_orders):
    index_names = ['Installations', 'Number of orders', 'Fleet_size', 'Installation ordering', 'Weather scenario']
    column_names = ['Value']
    df = pd.DataFrame([[number_of_insts], [number_of_orders], [fleet_size], [inst_ordering], 
                       [weather_scenario]], columns=column_names, index=index_names)
    return df

def organize_order_composition(order_composition):
    column_names = ['Order', 'Size', 'Installation', 'Node']
    rows = []
    for order in order_composition.keys():
        order_type = order_composition[order]['order']
        size = order_composition[order]['size']
        installation = order_composition[order]['installation']
        node = order_composition[order]['node']
        row = [order_type, size, installation, node]
        rows.append(row)
    df = pd.DataFrame(rows, columns=column_names)
    return df

def define_map():
    middle = [60.793142, 3.601824]
    m = folium.Map(location=middle,
                   zoom_start=8,
                   zoom_control=False)
    folium.TileLayer('cartodbpositron').add_to(m)
    return m

def add_markers_and_legs(m, voyage_by_insts, vessel_to_color, inst_to_coord):
    
    for index, row in voyage_by_insts.iterrows():
        vessel = int(row['Vessel'])
        start_inst, end_inst = int(row['Start inst']), int(row['End inst'])
        if start_inst == 0 and end_inst == 0:
            continue
        color = vessel_to_color[vessel]
        folium.Marker(location=inst_to_coord[start_inst], 
                      icon=folium.DivIcon(html=f"""<div style="font-family: courier new; font-size: 30px; color: green">{start_inst}</div>""")).add_to(m)
        folium.Marker(location=inst_to_coord[end_inst],
                      icon=folium.DivIcon(html=f"""<div style="font-family: courier new; font-size: 30px; color: green">{end_inst}</div>""")).add_to(m)
        folium.PolyLine(locations=[inst_to_coord[start_inst],
                                   inst_to_coord[end_inst]],
                                   color=color, weight=2).add_to(m)

In [6]:
with open(file_path) as results:
    json_file = json.load(results)
    
voyages = json_file['voyages']

model_runtime = json_file['runtime']['model_runtime']
preprocess_runtime = json_file['runtime']['preprocess_runtime']

fuel_costs = json_file['objective']['fuel_costs']
charter_costs = json_file['objective']['charter_costs']
penalty_costs = json_file['objective']['penalty_costs']

postponed_orders = json_file['order_fulfillment']['postponed_orders']
serviced_orders = json_file['order_fulfillment']['serviced_orders']
number_of_orders = len(postponed_orders) + len(serviced_orders)
end_depot_node = number_of_orders + 1

fleet_size = json_file['instance_info']['fleet_size']
inst_ordering = json_file['instance_info']['installation_ordering']
number_of_insts = json_file['instance_info']['number_of_installations']
order_composition = json_file['instance_info']['order_composition']
weather_scenario = json_file['instance_info']['weather_scenario']

In [7]:
instance_info = organize_instance_info(fleet_size, inst_ordering, number_of_insts, 
                                       weather_scenario, number_of_orders)
order_info = organize_order_composition(order_composition)
voyage_by_orders = summarize_voyage_orders(voyages)
voyage_by_insts = summarize_voyage_insts(voyage_by_orders, order_info, end_depot_node)
solution_info = organize_solution_attributes(postponed_orders, serviced_orders, fuel_costs, 
                                             charter_costs, penalty_costs, model_runtime, preprocess_runtime)

In [8]:
instance_info

,Value
Installations,4
Number of orders,4
Fleet_size,1
Installation ordering,Random
Weather scenario,0


In [9]:
order_info

,Order,Size,Installation,Node
0,MD_VFB,11.25,18,1
1,MD_VMO,15.00,16,2
2,OD_TRO,15.00,1,3
3,OP_HUL,5.00,20,4


In [10]:
voyage_by_orders

,Vessel,Start node,End node,Start time,Arrival time,End time,Delivery load,Pickup load,Arc fuel cost,Arc charter cost
0,0,0,1,63,86,91,41.25,0.0,850.263931,0.0
1,0,1,4,91,94,96,30.00,0.0,117.053518,0.0
2,0,4,2,96,101,107,30.00,5.0,260.083448,0.0
3,0,2,3,107,125,131,15.00,5.0,562.037106,0.0
4,0,3,5,131,148,148,0.00,5.0,435.820500,0.0
5,1,0,5,63,63,63,0.00,0.0,0.000000,0.0


In [11]:
voyage_by_insts

,Vessel,Start inst,End inst,Start time,Arrival time,End time,Delivery load,Pickup load,Arc fuel cost,Arc charter cost
0,0,1,0,131,148,148,0.00,5.0,435.820500,0.0
1,0,16,1,107,125,131,15.00,5.0,562.037106,0.0
2,0,20,16,96,101,107,30.00,5.0,260.083448,0.0
3,0,0,18,63,86,91,41.25,0.0,850.263931,0.0
4,0,18,20,91,94,96,30.00,0.0,117.053518,0.0
5,1,0,0,63,63,63,0.00,0.0,0.000000,0.0


In [12]:
solution_info

,Value
Postponed orders,[]
Serviced orders,"[1, 2, 3, 4]"
Fuel costs,2225.26
Charter costs,0
Penalty costs,0
Model runtime,0.0297983
Preprocess runtime,0.247975


In [2]:
def create_speed_profile(vessel, voyage_by_insts):
    fig = go.Figure()

    X, Y = [], []

    sub_df = voyage_by_insts.loc[voyage_by_insts['Vessel'] == vessel]
    if len(sub_df) == 1:
        return
    start_times, arr_times = sub_df['Start time'], sub_df['Arrival time']
    start_insts, end_insts = sub_df['Start inst'], sub_df['End inst']
    speeds = sub_df['Speed']
    for start_time, arr_time, start_inst_idx, end_inst_idx, speed in zip(start_times, arr_times, start_insts, end_insts, speeds):
        start_inst, end_inst = data.INSTALLATIONS[start_inst_idx], data.INSTALLATIONS[end_inst_idx]

        X.append([tp for tp in range(start_time, arr_time)])
        Y.append([speed] * (arr_time - start_time))

    for x, y in zip(X, Y):
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='royalblue', width=2)))

    flat_X = [x_coord for sub_x in X for x_coord in sub_x]
    flat_X.sort(reverse=True)
    lowest_speed = [7 for _ in range(len(flat_X))]
    eight_speed = [8 for _ in range(len(flat_X))]
    nine_speed = [9 for _ in range(len(flat_X))]
    ten_speed = [10 for _ in range(len(flat_X))]
    eleven_speed = [11 for _ in range(len(flat_X))]
    twelve_speed = [12 for _ in range(len(flat_X))]
    thirteen_speed = [13 for _ in range(len(flat_X))]
    highest_speed = [14 for _ in range(len(flat_X))]

    fig.add_trace(go.Scatter(x=flat_X, y=lowest_speed, mode='lines', line=dict(color='firebrick', width=1), showlegend=False))
    fig.add_trace(go.Scatter(x=flat_X, y=eight_speed, mode='lines', line=dict(color='tan', width=1, dash='dot'), showlegend=False))
    fig.add_trace(go.Scatter(x=flat_X, y=nine_speed, mode='lines', line=dict(color='tan', width=1, dash='dot'), showlegend=False))
    fig.add_trace(go.Scatter(x=flat_X, y=ten_speed, mode='lines', line=dict(color='tan', width=1, dash='dot'), showlegend=False))
    fig.add_trace(go.Scatter(x=flat_X, y=eleven_speed, mode='lines', line=dict(color='tan', width=1, dash='dot'), showlegend=False))
    fig.add_trace(go.Scatter(x=flat_X, y=twelve_speed, mode='lines', line=dict(color='tan', width=1, dash='dot'), showlegend=False))
    fig.add_trace(go.Scatter(x=flat_X, y=thirteen_speed, mode='lines', line=dict(color='tan', width=1, dash='dot'), showlegend=False))
    fig.add_trace(go.Scatter(x=flat_X, y=highest_speed, mode='lines', line=dict(color='firebrick', width=1), showlegend=False))

    fig.update_layout(yaxis=dict(showgrid=True,
                                 zeroline=True,
                                 showline=True,
                                 showticklabels=True),
                      plot_bgcolor='white')
    return fig

In [ ]:
figures = []
for vessel in range(fleet_size):
    figures.append(create_speed_profile(vessel, voyage_by_insts))

for figure in figures:
    figure.show()

In [14]:
m = define_map()

add_markers_and_legs(m, voyage_by_insts, vessel_to_color, inst_to_coord)

m